<a href="https://colab.research.google.com/github/fiscjona/myrepo/blob/main/sparql_queries_actors_viaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install SPARQLWrapper


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 10.2 MB/s eta 0:00:00


In [5]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Load the CSV file
authors_df = pd.read_csv('missing_viaf.csv')

# Initialize the SPARQL endpoint
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Function to execute a SPARQL query and return results
def execute_sparql_query(viaf_id):
    query = f"""
    SELECT ?author ?authorLabel ?viaf WHERE {{
      VALUES ?givenViaf {{ "{viaf_id}" }}
      ?author wdt:P214 ?givenViaf .
      ?author wdt:P214 ?viaf .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"]

# Iterate over each VIAF ID and execute the query
all_viaf_results = []

for index, row in authors_df.iterrows():
    viaf_id = row['viaf_link_numeric']
    results = execute_sparql_query(viaf_id)

    # Check if results are not empty
    if results:
        for result in results:
            viaf = result["viaf"]["value"]
            all_viaf_results.append({"name_unified": row['name_unified'], "viaf": viaf})
    else:
        # If no results, add the original VIAF ID with the author name
        all_viaf_results.append({"name_unified": row['name_unified'], "viaf": str(viaf_id)})

# Convert the results to a DataFrame
results_df = pd.DataFrame(all_viaf_results)

# Save the results to a new CSV file
results_df.to_csv('all_possible_viaf_ids.csv', index=False)
